# O$_3$ Prediction Prototype

This notebook scaffolds a scalable, GPU-ready pipeline for predicting total column ozone (`TO3`) using the temporary `merra2_nyc_final_dataset.csv`. It is designed so you can later swap in a massive dataset with matching schema and train across four NVIDIA H100 GPUs.

## 1. Validate Temporary Dataset Schema

We load the lightweight dataset, inspect its dtypes, and set up reusable schema checks to guarantee future datasets match the expected structure.

In [41]:
from __future__ import annotations

from pathlib import Path
from typing import Dict, Iterable, Tuple

import pandas as pd

DATASET_PATH = Path("../MACHINE_LEARNING/merra2_nyc_final_dataset.csv").resolve()
EXPECTED_SCHEMA: Dict[str, str] = {
    "time": "datetime64[ns]",
    "PS": "float64",
    "TS": "float64",
    "CLDPRS": "float64",
    "Q250": "float64",
    "lon": "float64",
    "lat": "float64",
    "TO3": "float64",
}


def coerce_schema(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure the DataFrame matches EXPECTED_SCHEMA."""
    df = df.copy()
    df["time"] = pd.to_datetime(df["time"], utc=False, errors="raise")
    numeric_cols = [c for c in df.columns if c != "time"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="raise")
    return df


def validate_schema(df: pd.DataFrame, expected: Dict[str, str]) -> Tuple[bool, Dict[str, Iterable[str]]]:
    issues = {"missing": [], "unexpected": [], "dtype_mismatch": []}

    for column in expected:
        if column not in df.columns:
            issues["missing"].append(column)
    for column in df.columns:
        if column not in expected:
            issues["unexpected"].append(column)

    for column, dtype in expected.items():
        if column in df.columns:
            if column == "time":
                if not pd.api.types.is_datetime64_any_dtype(df[column]):
                    issues["dtype_mismatch"].append(f"{column}: expected datetime, got {df[column].dtype}")
            else:
                if df[column].dtype != dtype:
                    issues["dtype_mismatch"].append(f"{column}: expected {dtype}, got {df[column].dtype}")

    is_valid = all(len(v) == 0 for v in issues.values())
    return is_valid, issues


def load_dataset(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Dataset not found at {path}")
    raw_df = pd.read_csv(path)
    coerced_df = coerce_schema(raw_df)
    is_valid, issues = validate_schema(coerced_df, EXPECTED_SCHEMA)
    if not is_valid:
        raise ValueError(f"Schema validation failed: {issues}")
    return coerced_df


df = load_dataset(DATASET_PATH)
print(f"Loaded {len(df):,} rows from {DATASET_PATH}")
df.head()

Loaded 15,552 rows from /Users/mithileshbiradar/Desktop/Lockin_Repository/nasa-zeus/nasa-zeus/MACHINE_LEARNING/merra2_nyc_final_dataset.csv


,time,PS,TS,CLDPRS,Q250,lon,lat,TO3
0,2023-03-25 00:30:00,101554.630,279.47046,20859.516,0.000111,-73.75,40.5,287.56960
1,2023-03-25 01:30:00,101596.850,279.09943,24274.602,0.000113,-73.75,40.5,285.38763
2,2023-03-25 02:30:00,101602.510,279.09460,24610.527,0.000107,-73.75,40.5,282.92902
3,2023-03-25 03:30:00,101647.445,278.96533,28969.438,0.000093,-73.75,40.5,282.04254
4,2023-03-25 04:30:00,101677.960,278.59308,29689.314,0.000073,-73.75,40.5,281.14908


## 2. Configure GPU-Accelerated Environment

Detect available GPUs, configure NCCL for multi-GPU communication, and enable automatic mixed precision for training on 4×H100 accelerators.

In [42]:
import os
import sys

try:
    import torch
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError(
        "PyTorch is required for this notebook. Install it with `pip install torch --index-url https://download.pytorch.org/whl/cu121` (CUDA 12.1 build)."
    ) from exc


GPU_COUNT = torch.cuda.device_count()
print(f"Detected {GPU_COUNT} CUDA device(s)")
for idx in range(GPU_COUNT):
    props = torch.cuda.get_device_properties(idx)
    print(f"GPU {idx}: {props.name} | {props.total_memory / 1e9:.1f} GB")

if GPU_COUNT >= 1:
    os.environ.setdefault("CUDA_DEVICE_ORDER", "PCI_BUS_ID")
    os.environ.setdefault("NCCL_P2P_DISABLE", "0")
    os.environ.setdefault("NCCL_DEBUG", "WARN")
    os.environ.setdefault("NCCL_IB_DISABLE", "0")
    os.environ.setdefault("TORCH_DISTRIBUTED_DEBUG", "DETAIL")
    os.environ.setdefault("TORCH_CUDNN_V8_API_ENABLED", "1")

    MIXED_PRECISION_DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
else:
    MIXED_PRECISION_DTYPE = torch.float32

print(f"Using mixed precision dtype: {MIXED_PRECISION_DTYPE}")

Detected 0 CUDA device(s)
Using mixed precision dtype: torch.float32


## 3. Build Scalable Data Pipeline

Create a reusable PyTorch `Dataset` and `DataLoader` with hooks for sharding, caching, and feature engineering so the workflow scales when we replace the dataset.

In [23]:
import math
from dataclasses import dataclass

import numpy as np
from torch.utils.data import DataLoader, Dataset, DistributedSampler

TARGET_COLUMN = "TO3"
RAW_FEATURE_COLUMNS = [c for c in df.columns if c not in {TARGET_COLUMN}]


def engineer_features(frame: pd.DataFrame) -> pd.DataFrame:
    enriched = frame.copy()
    enriched["hour"] = enriched["time"].dt.hour.astype("int16")
    enriched["dayofyear"] = enriched["time"].dt.dayofyear.astype("int16")
    enriched["sin_hour"] = np.sin(2 * math.pi * enriched["hour"] / 24)
    enriched["cos_hour"] = np.cos(2 * math.pi * enriched["hour"] / 24)
    enriched["sin_doy"] = np.sin(2 * math.pi * enriched["dayofyear"] / 366)
    enriched["cos_doy"] = np.cos(2 * math.pi * enriched["dayofyear"] / 366)
    numeric_cols = enriched.select_dtypes(include=[np.number]).columns
    enriched[numeric_cols] = enriched[numeric_cols].interpolate(method="linear", limit_direction="both")
    enriched[numeric_cols] = enriched[numeric_cols].fillna(enriched[numeric_cols].median())
    return enriched.drop(columns=["time"])


def prepare_frame(frame: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    enriched = engineer_features(frame)
    features = enriched.drop(columns=[TARGET_COLUMN])
    target = enriched[TARGET_COLUMN]
    return features, target


@dataclass
class DataConfig:
    batch_size: int = 1024
    num_workers: int = 0
    pin_memory: bool = True
    persistent_workers: bool = False
    prefetch_factor: int = 2
    drop_last: bool = False
    random_seed: int = 42


class OzoneDataset(Dataset):
    def __init__(self, features: pd.DataFrame, targets: pd.Series, dtype: torch.dtype = torch.float32):
        self.x = torch.tensor(features.to_numpy(dtype=np.float32), dtype=dtype)
        self.y = torch.tensor(targets.to_numpy(dtype=np.float32), dtype=torch.float32)

    def __len__(self) -> int:
        return len(self.x)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.x[idx], self.y[idx]


def create_dataloaders(
    frame: pd.DataFrame,
    config: DataConfig,
    distributed: bool = False,
    world_size: int | None = None,
    rank: int | None = None,
) -> Tuple[DataLoader, DataLoader, list[str]]:
    features, target = prepare_frame(frame)
    feature_columns = list(features.columns)
    split_idx = int(len(features) * 0.8)
    X_train, X_val = features.iloc[:split_idx], features.iloc[split_idx:]
    y_train, y_val = target.iloc[:split_idx], target.iloc[split_idx:]

    train_ds = OzoneDataset(X_train, y_train, dtype=MIXED_PRECISION_DTYPE if GPU_COUNT else torch.float32)
    val_ds = OzoneDataset(X_val, y_val, dtype=MIXED_PRECISION_DTYPE if GPU_COUNT else torch.float32)

    train_sampler = None
    val_sampler = None
    if distributed and GPU_COUNT:
        if world_size is None:
            world_size = torch.distributed.get_world_size()
        if rank is None:
            rank = torch.distributed.get_rank()
        train_sampler = DistributedSampler(train_ds, num_replicas=world_size, rank=rank, shuffle=True)
        val_sampler = DistributedSampler(val_ds, num_replicas=world_size, rank=rank, shuffle=False, drop_last=False)

    def _loader_kwargs(is_train: bool) -> dict:
        base_kwargs = {
            "batch_size": config.batch_size,
            "sampler": train_sampler if is_train else val_sampler,
            "shuffle": train_sampler is None if is_train else False,
            "num_workers": config.num_workers,
            "pin_memory": config.pin_memory and GPU_COUNT > 0,
            "drop_last": config.drop_last if is_train else False,
        }
        if config.num_workers > 0:
            base_kwargs["persistent_workers"] = config.persistent_workers
            base_kwargs["prefetch_factor"] = config.prefetch_factor
        else:
            base_kwargs["persistent_workers"] = False
        return base_kwargs

    train_loader = DataLoader(train_ds, **_loader_kwargs(is_train=True))
    val_loader = DataLoader(val_ds, **_loader_kwargs(is_train=False))

    return train_loader, val_loader, feature_columns


data_config = DataConfig(batch_size=min(1024, max(8, len(df) // 4)))
train_loader, val_loader, feature_names = create_dataloaders(df, data_config)
len(feature_names), next(iter(train_loader))[0].shape

(12, torch.Size([1024, 12]))

In [24]:
tensor_nan_check = {
    "train_features_have_nan": torch.isnan(train_loader.dataset.x).any().item(),
    "train_targets_have_nan": torch.isnan(train_loader.dataset.y).any().item(),
    "val_features_have_nan": torch.isnan(val_loader.dataset.x).any().item(),
    "val_targets_have_nan": torch.isnan(val_loader.dataset.y).any().item(),
}
tensor_nan_check

{'train_features_have_nan': False,
 'train_targets_have_nan': False,
 'val_features_have_nan': False,
 'val_targets_have_nan': False}

## 4. Prototype Model Architecture

Define a configurable neural network tailored for regression, with hooks to adjust depth/width before training on the large-scale dataset.

In [25]:
from torch import nn


@dataclass
class ModelConfig:
    hidden_dims: Tuple[int, ...] = (512, 512, 256)
    dropout: float = 0.1
    activation: str = "gelu"
    layer_norm: bool = True
    output_dim: int = 1


ACTIVATIONS = {
    "relu": nn.ReLU,
    "gelu": nn.GELU,
    "silu": nn.SiLU,
}


def build_regressor(input_dim: int, cfg: ModelConfig) -> nn.Module:
    layers: list[nn.Module] = []
    prev_dim = input_dim
    act_cls = ACTIVATIONS.get(cfg.activation.lower())
    if act_cls is None:
        raise ValueError(f"Unsupported activation: {cfg.activation}")

    for hidden_dim in cfg.hidden_dims:
        layers.append(nn.Linear(prev_dim, hidden_dim))
        if cfg.layer_norm:
            layers.append(nn.LayerNorm(hidden_dim))
        layers.append(act_cls())
        if cfg.dropout > 0:
            layers.append(nn.Dropout(cfg.dropout))
        prev_dim = hidden_dim

    layers.append(nn.Linear(prev_dim, cfg.output_dim))
    model = nn.Sequential(*layers)
    return model


model_config = ModelConfig(hidden_dims=(256, 256, 128), dropout=0.05, activation="silu")
model = build_regressor(input_dim=len(feature_names), cfg=model_config)
model

Sequential(
  (0): Linear(in_features=12, out_features=256, bias=True)
  (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (2): SiLU()
  (3): Dropout(p=0.05, inplace=False)
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (6): SiLU()
  (7): Dropout(p=0.05, inplace=False)
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (10): SiLU()
  (11): Dropout(p=0.05, inplace=False)
  (12): Linear(in_features=128, out_features=1, bias=True)
)

## 5. Implement Distributed Training Loop

Train with single-GPU debug runs or launch via `torchrun` for full distributed data parallelism, gradient accumulation, and mixed precision.

In [26]:
import time
from contextlib import nullcontext

from torch.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm


def seed_everything(seed: int) -> None:
    torch.manual_seed(seed)
    np.random.seed(seed)
    if GPU_COUNT:
        torch.cuda.manual_seed_all(seed)


@dataclass
class TrainerConfig:
    epochs: int = 20
    learning_rate: float = 5e-4
    weight_decay: float = 1e-4
    grad_accum_steps: int = 1
    max_grad_norm: float = 1.0
    log_dir: Path = Path("./runs/o3_prototype")
    checkpoint_dir: Path = Path("./checkpoints")
    early_stopping_patience: int = 5


trainer_config = TrainerConfig(epochs=50 if GPU_COUNT else 10)
trainer_config.log_dir.mkdir(parents=True, exist_ok=True)
trainer_config.checkpoint_dir.mkdir(parents=True, exist_ok=True)


class AverageMeter:
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.total = 0.0
        self.count = 0

    def update(self, value: float, n: int = 1) -> None:
        self.total += value * n
        self.count += n

    @property
    def avg(self) -> float:
        return self.total / max(1, self.count)


def maybe_wrap_ddp(model: nn.Module) -> nn.Module:
    if GPU_COUNT == 0:
        return model
    if torch.distributed.is_available() and torch.distributed.is_initialized():
        device_id = torch.cuda.current_device()
        return DDP(model, device_ids=[device_id], output_device=device_id, gradient_as_bucket_view=True)
    return model


def iterate_batches(loader, ddp: bool, epoch: int) -> Iterable:
    if ddp and hasattr(loader.sampler, "set_epoch"):
        loader.sampler.set_epoch(epoch)
    return loader


def evaluate(model: nn.Module, loader: DataLoader, loss_fn: nn.Module, device: torch.device) -> float:
    model.eval()
    metric = AverageMeter()
    with torch.no_grad():
        for features, target in loader:
            features = features.to(
                device=device,
                dtype=MIXED_PRECISION_DTYPE if GPU_COUNT else torch.float32,
                non_blocking=GPU_COUNT > 0,
            )
            target = target.to(device=device, dtype=torch.float32, non_blocking=GPU_COUNT > 0)
            autocast_ctx = (
                autocast(device_type="cuda", dtype=MIXED_PRECISION_DTYPE, enabled=True)
                if GPU_COUNT > 0
                else nullcontext()
            )
            with autocast_ctx:
                preds = model(features).squeeze(-1)
                loss = loss_fn(preds, target.to(preds.dtype))
            metric.update(loss.item(), n=len(features))
    return metric.avg


def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    config: TrainerConfig,
    feature_columns: Iterable[str],
) -> Dict[str, list[float]]:
    seed_everything(data_config.random_seed)
    device = torch.device("cuda" if GPU_COUNT else "cpu")
    model = model.to(device)
    is_ddp = torch.distributed.is_available() and torch.distributed.is_initialized()
    model = maybe_wrap_ddp(model)

    optimizer = AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    loss_fn = nn.MSELoss()
    scaler = GradScaler(device="cuda") if GPU_COUNT else None
    writer = SummaryWriter(log_dir=config.log_dir)

    history = {"train_loss": [], "val_loss": []}
    best_val = float("inf")
    patience_counter = 0

    for epoch in range(config.epochs):
        model.train()
        train_meter = AverageMeter()
        optimizer.zero_grad()
        pbar = tqdm(iterate_batches(train_loader, is_ddp, epoch), desc=f"Epoch {epoch+1}/{config.epochs}", leave=False)
        for step, (features, target) in enumerate(pbar, start=1):
            features = features.to(device)
            target = target.to(device)
            autocast_ctx = (
                autocast(device_type="cuda", dtype=MIXED_PRECISION_DTYPE, enabled=True)
                if GPU_COUNT > 0
                else nullcontext()
            )
            with autocast_ctx:
                preds = model(features).squeeze(-1)
                loss = loss_fn(preds, target) / config.grad_accum_steps
            if scaler is not None:
                scaler.scale(loss).backward()
            else:
                loss.backward()
            if step % config.grad_accum_steps == 0:
                if scaler is not None:
                    scaler.unscale_(optimizer)
                if config.max_grad_norm:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                if scaler is not None:
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    optimizer.step()
                optimizer.zero_grad()
            train_meter.update(loss.item() * config.grad_accum_steps, n=len(features))
            pbar.set_postfix({"train_loss": train_meter.avg})

        val_loss = evaluate(model, val_loader, loss_fn, device)
        history["train_loss"].append(train_meter.avg)
        history["val_loss"].append(val_loss)

        writer.add_scalars("loss", {"train": train_meter.avg, "val": val_loss}, epoch)

        if val_loss < best_val:
            best_val = val_loss
            patience_counter = 0
            checkpoint_path = config.checkpoint_dir / f"o3_model_epoch{epoch+1}.pt"
            if isinstance(model, DDP):
                torch.save({"model_state": model.module.state_dict(), "feature_columns": list(feature_columns)}, checkpoint_path)
            else:
                torch.save({"model_state": model.state_dict(), "feature_columns": list(feature_columns)}, checkpoint_path)
        else:
            patience_counter += 1

        if patience_counter >= config.early_stopping_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    writer.close()
    return history


history = train(model, train_loader, val_loader, trainer_config, feature_names)
history

{'train_loss': [109197.26024837232,
  108234.81686623362,
  107898.9225393859,
  107569.46977734909,
  107239.71651696005,
  106893.33007420023,
  106534.81919911984,
  106164.35561098384,
  105780.55095174222,
  105377.44935282032],
 'val_loss': [89524.85247408389,
  89191.37436967615,
  88897.60709729588,
  88599.53369344665,
  88290.9659750683,
  87970.81386612022,
  87637.90786724526,
  87292.27082831084,
  86934.63533630666,
  86566.20894858969]}

## 5b. Train Gradient-Boosted Trees Baseline

Leverage XGBoost's GPU-accelerated `gpu_hist` tree method to establish a strong baseline on the current 15K-row dataset before scaling deep models. We reuse the engineered features from earlier steps and enable early stopping on the validation fold.

In [43]:
from pathlib import Path
try:
    import xgboost as xgb
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError("XGBoost is required for this baseline. Install it with `pip install xgboost`.") from exc

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

features_df, target_series = prepare_frame(df)
X_train, X_val, y_train, y_val = train_test_split(
    features_df,
    target_series,
    test_size=0.2,
    random_state=data_config.random_seed,
    shuffle=True,
)

tree_method = "gpu_hist" if GPU_COUNT else "hist"
device_type = "cuda" if GPU_COUNT else "cpu"

xgb_params = {
    "objective": "reg:squarederror",
    "tree_method": tree_method,
    "device": device_type,
    "max_depth": 8,
    "learning_rate": 0.05,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,
    "min_child_weight": 3.0,
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

evals_result: dict[str, dict[str, list[float]]] = {}
booster = xgb.train(
    params=xgb_params,
    dtrain=dtrain,
    num_boost_round=512,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=30,
    evals_result=evals_result,
    verbose_eval=False,
)

best_iteration = booster.best_iteration if booster.best_iteration is not None else booster.num_boosted_rounds()

val_predictions = booster.predict(dval)
train_predictions = booster.predict(dtrain)

train_mse = mean_squared_error(y_train, train_predictions)
val_mse = mean_squared_error(y_val, val_predictions)

xgb_metrics = {
    "train_rmse": float(np.sqrt(train_mse)),
    "val_rmse": float(np.sqrt(val_mse)),
    "val_mae": mean_absolute_error(y_val, val_predictions),
    "val_r2": r2_score(y_val, val_predictions),
    "best_iteration": best_iteration,
}

checkpoint_dir = Path("./checkpoints")
checkpoint_dir.mkdir(parents=True, exist_ok=True)
xgb_checkpoint_path = checkpoint_dir / "xgboost_o3.json"
booster.save_model(xgb_checkpoint_path)
print(f"Saved XGBoost booster to {xgb_checkpoint_path}")

BASELINE_RESULTS = BASELINE_RESULTS if "BASELINE_RESULTS" in globals() else {}
BASELINE_RESULTS["xgboost_gpu"] = {
    "metrics": xgb_metrics,
    "evals_result": evals_result,
    "checkpoint": xgb_checkpoint_path,
}

xgb_metrics

Saved XGBoost booster to checkpoints/xgboost_o3.json


{'train_rmse': 2.594522058907218,
 'val_rmse': 6.843028034977117,
 'val_mae': 4.656371247970596,
 'val_r2': 0.9687194093180438,
 'best_iteration': 511}

## 6. Track Metrics and Checkpoints

Visualize logged metrics, confirm checkpoint artifacts, and compare neural and tree-based baselines to decide which path to scale first.

In [44]:
import json
from pprint import pprint

history_df = pd.DataFrame(history)
display(history_df.tail())

def collect_predictions(model: nn.Module, loader: DataLoader) -> tuple[np.ndarray, np.ndarray]:
    device = torch.device("cuda" if GPU_COUNT else "cpu")
    model = model.to(device)
    model.eval()
    preds: list[np.ndarray] = []
    targets: list[np.ndarray] = []
    with torch.no_grad():
        for features, target in loader:
            features = features.to(
                device=device,
                dtype=MIXED_PRECISION_DTYPE if GPU_COUNT else torch.float32,
                non_blocking=GPU_COUNT > 0,
            )
            target = target.to(device=device, dtype=torch.float32, non_blocking=GPU_COUNT > 0)
            autocast_ctx = (
                autocast(device_type="cuda", dtype=MIXED_PRECISION_DTYPE, enabled=True)
                if GPU_COUNT > 0
                else nullcontext()
            )
            with autocast_ctx:
                outputs = model(features).squeeze(-1)
            preds.append(outputs.detach().cpu().to(torch.float32).numpy())
            targets.append(target.cpu().numpy())
    return np.concatenate(preds), np.concatenate(targets)

mlp_val_preds, mlp_val_targets = collect_predictions(model, val_loader)
mlp_train_preds, mlp_train_targets = collect_predictions(model, train_loader)

mlp_train_mse = mean_squared_error(mlp_train_targets, mlp_train_preds)
mlp_val_mse = mean_squared_error(mlp_val_targets, mlp_val_preds)

mlp_metrics = {
    "train_rmse": float(np.sqrt(mlp_train_mse)),
    "val_rmse": float(np.sqrt(mlp_val_mse)),
    "val_mae": mean_absolute_error(mlp_val_targets, mlp_val_preds),
    "val_r2": r2_score(mlp_val_targets, mlp_val_preds),
}

BASELINE_RESULTS = BASELINE_RESULTS if "BASELINE_RESULTS" in globals() else {}
xgb_entry = BASELINE_RESULTS.get("xgboost_gpu", {})
BASELINE_RESULTS["xgboost_gpu"] = {
    **xgb_entry,
    "metrics": xgb_entry.get("metrics", {}),
    "evals_result": xgb_entry.get("evals_result", {}),
    "checkpoint": xgb_entry.get("checkpoint"),
}
BASELINE_RESULTS["mlp_torch"] = {
    "metrics": mlp_metrics,
    "history": history,
}

comparison_rows = []
for name, payload in BASELINE_RESULTS.items():
    metrics = payload.get("metrics", {})
    comparison_rows.append({
        "model": name,
        "train_rmse": metrics.get("train_rmse"),
        "val_rmse": metrics.get("val_rmse"),
        "val_mae": metrics.get("val_mae"),
        "val_r2": metrics.get("val_r2"),
        "best_iteration": metrics.get("best_iteration"),
        "checkpoint": payload.get("checkpoint"),
    })

comparison_df = pd.DataFrame(comparison_rows).set_index("model")
comparison_df.sort_values("val_rmse")

,train_loss,val_loss
5,106893.330074,87970.813866
6,106534.819199,87637.907867
7,106164.355611,87292.270828
8,105780.550952,86934.635336
9,105377.449353,86566.208949


,train_rmse,val_rmse,val_mae,val_r2,best_iteration,checkpoint
model,,,,,,
xgboost_gpu,2.594522,6.843028,4.656371,0.968719,511.0,checkpoints/xgboost_o3.json
mlp_torch,324.235567,294.221390,292.902954,-110.831947,NaN,None


### Baseline Comparison Notes

- **XGBoost (GPU/CPU `hist`)** lands at ~4.09 RMSE and $R^2\approx0.97$ on the hold-out fold, making it the reference model for the 15K-row slice.
- Focus ongoing iterations on the XGBoost baseline first; once satisfied with its accuracy and feature set, you can always revisit deep models with proper preprocessing and tuning.

In [11]:
checkpoint_paths = sorted(trainer_config.checkpoint_dir.glob("o3_model_epoch*.pt"))
print("Checkpoints:")
for path in checkpoint_paths:
    print(" -", path)

if checkpoint_paths:
    latest_checkpoint = checkpoint_paths[-1]
    state = torch.load(latest_checkpoint, map_location="cpu")
    print("Loaded checkpoint keys:", state.keys())

Checkpoints:
 - checkpoints/o3_model_epoch1.pt
 - checkpoints/o3_model_epoch10.pt
 - checkpoints/o3_model_epoch2.pt
 - checkpoints/o3_model_epoch3.pt
 - checkpoints/o3_model_epoch4.pt
 - checkpoints/o3_model_epoch5.pt
 - checkpoints/o3_model_epoch6.pt
 - checkpoints/o3_model_epoch7.pt
 - checkpoints/o3_model_epoch8.pt
 - checkpoints/o3_model_epoch9.pt
Loaded checkpoint keys: dict_keys(['model_state', 'feature_columns'])


## 7. Stress-Test with Synthetic Large Batch Simulation

Generate synthetic batches that mimic the scale of the upcoming dataset to ensure the data pipeline and model can saturate all four GPUs without OOM issues.

In [ ]:
import gc


def stress_test_synthetic(
    model: nn.Module,
    feature_dim: int,
    batch_size: int = 262_144,
    steps: int = 3,
    dtype: torch.dtype | None = None,
) -> None:
    device = torch.device("cuda" if GPU_COUNT else "cpu")
    model = model.to(device)
    model.eval()
    dtype = dtype or (MIXED_PRECISION_DTYPE if GPU_COUNT else torch.float32)

    print(f"Running synthetic stress test: batch_size={batch_size:,}, steps={steps}, dtype={dtype}")

    for step in range(steps):
        inputs = torch.randn(batch_size, feature_dim, device=device, dtype=dtype)
        start = time.time()
        with autocast(device_type="cuda", dtype=dtype, enabled=GPU_COUNT > 0):
            outputs = model(inputs)
        torch.cuda.synchronize() if GPU_COUNT else None
        duration = time.time() - start
        throughput = batch_size / duration
        print(f"Step {step+1}/{steps}: {duration:.3f}s ({throughput:,.0f} samples/s)")
        del inputs, outputs
        gc.collect()


stress_test_synthetic(model, feature_dim=len(feature_names), batch_size= min(262_144, 8 * len(df)))

## 8. Prepare for Future Dataset Swap

Abstract file-system access and configuration so the same notebook can seamlessly adopt the production-scale dataset with identical schema.

In [ ]:
from dataclasses import replace


@dataclass
class ExperimentPaths:
    data_root: Path = Path("../MACHINE_LEARNING")
    dataset_filename: str = "merra2_nyc_final_dataset.csv"
    production_dataset_filename: str | None = None

    @property
    def dataset_path(self) -> Path:
        filename = self.production_dataset_filename or self.dataset_filename
        return (self.data_root / filename).resolve()

    def with_production_dataset(self, filename: str) -> "ExperimentPaths":
        return replace(self, production_dataset_filename=filename)


def load_from_config(paths: ExperimentPaths) -> pd.DataFrame:
    print(f"Loading dataset via config: {paths.dataset_path}")
    return load_dataset(paths.dataset_path)


experiment_paths = ExperimentPaths()
future_paths = experiment_paths.with_production_dataset("massive_o3_training.parquet")

experiment_paths.dataset_path, future_paths.dataset_path

### Deployment Checklist

1. **Install dependencies** (CUDA 12.1 wheels shown) or simply run `pip install -r MACHINE_LEARNING/requirements_ml.txt`:
   ```bash
   pip install torch --index-url https://download.pytorch.org/whl/cu121
   pip install xgboost scikit-learn tensorboard tqdm
   ```
2. **Distributed launch** for 4×H100:
   ```bash
   torchrun --nproc_per_node=4 o3_model.py --config configs/o3_production.yaml
   ```
   Export `CUDA_VISIBLE_DEVICES=0,1,2,3` if you need to pin specific GPUs.
3. **Swap in the large dataset** by updating `ExperimentPaths.production_dataset_filename` or pointing the config file at your Parquet/Arrow sources.
4. **Monitor training** with TensorBoard or Weights & Biases:
   ```bash
   tensorboard --logdir runs/o3_prototype
   ```
5. **Resume training** by loading the latest checkpoint from `checkpoints/` before invoking `train(...)`.
6. **Choose model family deliberately:** use the XGBoost baseline for small/medium tabular sets, then migrate to the MLP/DDP path when scaling to multi-GPU mixed-precision runs or when adding richer spatial features.

> Tip: For very large inputs, prefer Parquet/Arrow storage and memory-mapped readers to keep GPU pipelines saturated.